In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data = pd.read_csv('./Churn_Modelling.csv')
data.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


In [2]:
X = data.iloc[:, 3:13]
y = data.iloc[:, 13]

In [3]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
X['Geography'] = label_encoder.fit_transform(X['Geography'])
X['Gender'] = label_encoder.fit_transform(X['Gender'])

#geo = pd.get_dummies(X['Geography'],drop_first=True)
#gen = pd.get_dummies(X['Gender'], drop_first=True)

In [4]:
#Geography = geo.copy()
#Gender = gen.copy()

#X = X.drop(['Gender','Geography'], axis = 1)
#X = pd.concat([X,Gender, Geography],axis=1)

# Female - 0, Male - 1
#France - 0, Germany - 1, Spain - 2

X.head(10)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,0,0,42,2,0.00,1,1,1,101348.88
1,608,2,0,41,1,83807.86,1,0,1,112542.58
2,502,0,0,42,8,159660.80,3,1,0,113931.57
3,699,0,0,39,1,0.00,2,0,0,93826.63
4,850,2,0,43,2,125510.82,1,1,1,79084.10
5,645,2,1,44,8,113755.78,2,1,0,149756.71
6,822,0,1,50,7,0.00,2,1,1,10062.80
7,376,1,0,29,4,115046.74,4,1,0,119346.88
8,501,0,1,44,4,142051.07,2,0,1,74940.50
9,684,0,1,27,2,134603.88,1,1,1,71725.73


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [6]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout

Using TensorFlow backend.


In [7]:
tuned_classifier = Sequential()

tuned_classifier.add(Dense(units=40, kernel_initializer='he_uniform', activation='relu', input_dim=X_train.shape[1]))
tuned_classifier.add(Dropout(0.3))

tuned_classifier.add(Dense(units=20, kernel_initializer = 'he_uniform', activation='relu'))
tuned_classifier.add(Dropout(0.3))

tuned_classifier.add(Dense(units=1, kernel_initializer='glorot_uniform', activation='sigmoid'))

tuned_classifier.compile(optimizer='Adamax', loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
tuned_model_history = tuned_classifier.fit(X_train, y_train, validation_split=0.33, batch_size=128, epochs=40)

Train on 5359 samples, validate on 2641 samples
Epoch 1/40
5359/5359 [==============================] - 0s 46us/step - loss: 0.7035 - accuracy: 0.6604 - val_loss: 0.5163 - val_accuracy: 0.7955
Epoch 2/40
5359/5359 [==============================] - 0s 14us/step - loss: 0.5767 - accuracy: 0.7462 - val_loss: 0.4918 - val_accuracy: 0.7993
Epoch 3/40
5359/5359 [==============================] - 0s 15us/step - loss: 0.5420 - accuracy: 0.7597 - val_loss: 0.4800 - val_accuracy: 0.8008
Epoch 4/40
5359/5359 [==============================] - 0s 16us/step - loss: 0.5303 - accuracy: 0.7709 - val_loss: 0.4719 - val_accuracy: 0.8016
Epoch 5/40
5359/5359 [==============================] - 0s 15us/step - loss: 0.5211 - accuracy: 0.7748 - val_loss: 0.4667 - val_accuracy: 0.8023
Epoch 6/40
5359/5359 [==============================] - 0s 19us/step - loss: 0.5185 - accuracy: 0.7766 - val_loss: 0.4625 - val_accuracy: 0.8039
Epoch 7/40
5359/5359 [==============================] - 0s 17us/step - loss: 0.495

In [13]:
from sklearn.metrics import accuracy_score
y_pred = tuned_classifier.predict(X_test)
y_pred = (y_pred>0.5)
tuned_score = accuracy_score(y_pred, y_test)
print("Score after Tuning the model is : "+str(tuned_score))

Score after Tuning the model is : 0.857


In [14]:

filename = 'churn-model-85'
import pickle
pickle.dump(model, open(filename, 'wb'))
print("Model saved succesfully!")

NameError: name 'model' is not defined